# KASSANDR - GRUPO 5
- Participantes:
    - Elias Tolentino
    - Gabriel Guedes

+ KASANDR é uma coleção de dados para sistema de recomendações que recorda o comportamento dos clientes do site de e-commerce Kelkoo na Europa nos países: Alemanha, França e Itália.
+ Foi usado um sample de 1mi de linhas dos clientes da alemanha para modelagem e analise desta apresentação

In [ ]:
#IMPORT DE BIBLIOTECAS

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix

In [ ]:
df_de_cat = pd.read_csv('category_de.txt') # DataFrame Informativo das categorias

In [ ]:
df_de_cat.head()

#### O ANCESTOR ID REÚNE, DA ESQUERDA PARA A DIREITA, ID DO PRODUTO -> HIERARQUIA DE MACROCATEGORIAS

## Tratamento de clicks

In [ ]:
clicks_de = pd.read_csv('clicks_de_sample_2.txt', sep = ',', header=0) # df de clicks

In [ ]:
clicks_de.drop(columns = ['Keywords'], axis = 1, inplace=True)
#clicks_de['Category'] = clicks_de['Category'].astype(str)
pontuacao_de = pd.DataFrame(clicks_de['Category'].value_counts()).reset_index()
pontuacao_de.columns = ['Category', 'Pontuacao']
dici = dict(zip(pontuacao_de.Category, pontuacao_de.Pontuacao))
clicks_de['Cat_clicks'] = clicks_de['Category'].map(dici)
clicks_de.sort_values(by='Cat_clicks', inplace = True)

In [ ]:
## RENOMEANDO OFERTAS E USUÁRIOS
clicks_de['#User'] = clicks_de['UserId'].map(dict(zip(clicks_de['UserId'].unique(), 
                                                                        range(len(clicks_de['UserId']))))) # Redefinindo indices para usuário apenas como numérico

clicks_de['#Offer'] = clicks_de['OfferTitle'].map(dict(zip(clicks_de['OfferTitle'].unique(), 
                                                                        range(len(clicks_de['OfferTitle']))))) #  Redefinindo indices para ofertas apenas como numérico considerando o OfferTitle

In [ ]:
clicks_de = clicks_de.merge(df_de_cat, left_on = 'Category', right_on = 'ID').drop('ID', axis = 1) # Adicionar 

## Classificação das categorias e ofertas mais buscadas

In [ ]:
categoria_buscas = clicks_de.groupby('Name')['Name'].count() 
categoria_buscas.sort_values(ascending = False) # Categorias mais buscadas

In [ ]:
produtos_buscas = clicks_de.groupby('OfferTitle')['Name'].count() 
produtos_buscas.sort_values(ascending = False) # Ofertas mais buscadas

In [ ]:
clicks_de = clicks_de.merge(categoria_buscas, left_on = 'Name', right_index = True).drop('Name_x',axis = 1).rename({'Name_y': 'CategoryClicks'}, axis = 1) # Adição de coluna de Clicks por categoria
clicks_de.drop(columns = 'Cat_clicks', inplace = True)

In [ ]:
clicks_de = clicks_de.merge(produtos_buscas, left_on = 'OfferTitle', right_index = True).drop('Name_x',axis = 1).rename({'Name_y': 'ProductClicks'}, axis = 1)  # Adição de coluna de clicks por produto/Offer

In [ ]:
clicks_de['UserClicks'] = clicks_de['UserId'].map(dict(zip(clicks_de['UserId'].value_counts().index,
                                                           clicks_de['UserId'].value_counts().values))) #Criação de coluna com nº total de cliques do usuário

In [ ]:
clicks_de_filtered = clicks_de[(clicks_de.UserClicks > 10)] # Filtro de usuários com 10 ou mais Clicks

In [ ]:
clicks_de_filtered

In [ ]:
user_item_clicks = clicks_de_filtered[['#User','#Offer','UserClicks']]
#user_item_clicks.UserClicks[user_item_clicks.UserClicks != 0] = 1
user_item_clicks.to_csv('user_item_clicks', index = False, header = False)

# Criação da matriz esparsa para avaliação da correlação entre usuários
- Para o sistema de recomendação, optou-se por um sitema de filtragem colaborativa, inicialmente com a abordagem Memory-based e User-based
- Matriz formada a partir dos usuarios nas colunas
- Como o dataset conta apenas com os cliques do usuario, considerou-se que seria mais coerente uma analise de coerrelação avaliando a similaridade de cliques entre usuários

In [ ]:
sparse = clicks_de_filtered.pivot_table(index = '#Offer', columns = '#User', values = 'UserClicks') # Matriz criada considerando usuários nas colunas.
sparse # Como o dataset conta apenas com os cliques dados pelo usuário, considerou-se que seria mais coerente uma análise de correlação avaliando a similaridade de clicks entre usuários

In [ ]:
sparse[sparse.isna()] = 0
sparse[sparse != 0] = 1 # O número de clicks de um usuário em uma mesma oferta foi considerado irrelevante. Considerou-se apenas se o usuário clicou ou não na oferta.

In [ ]:
sparse.head()

## Dicionário de Ofertas

In [ ]:
clicks_de_filtered[['OfferTitle', '#Offer']] 

In [ ]:
offer_title = dict(zip(clicks_de_filtered['#Offer'].unique(), clicks_de_filtered['OfferTitle'].unique())) #Criação do dicionário que relaciona o nome da oferta (OfferTitle) com o código #Offer para tradução da recomendação dada

# SELECAO DE USUARIOS PARECIDOS COM O USUARIO SELECIONADO E RECOMENDAÇÕES
- Função que recebe o código das recomendações do usuário selecionado e retorna os nomes das ofertas (Parametro: clicks do usuário)
- Função que calcula as recomendações para um usuário (user) qualquer e retorna o top5 de ofertas mais relevantes

In [ ]:
#Função que recebe o código das recomendações do usuário selecionado e retorna os nomes das ofertas, considerando os clicks do usuário (segundo parametro)
def result_final(recom: pd.Series, clicks_selected_user): 
    print('Clicks do usuário: ')  #Função que complementa a função lista_recomendacoes()
    for item in clicks_selected_user:
        print(offer_title[item])
    print(' ')
    print('Recomendações: ')
    for item in recom.index:
        #print(item)
        print(offer_title[item])

In [ ]:
def lista_recomendacoes(user): #Função que calcula as recomendações para um usuário user qualquer e retorna o top5 de ofertas mais relevantes
    
    # Seleção de usuários com maior correlação com o usuário selecionado
    topmatch_users = (sparse.corrwith(sparse[user]).sort_values(ascending = False))[1:]
    clicks_selected_user = clicks_de_filtered[clicks_de_filtered['#User']==user]['#Offer'].drop_duplicates().values
    
    # Os weights indica quanto aquele novo produto é relevante para o usuário em relação ao nível de correlação entre usuários
    weights = topmatch_users.values
    
    recommendations = [] # Lista de possíveis indicações com base nos clicks dos usuários semelhantes
    for user_id in range(len(topmatch_users[:5])): # Loop pelos usuários mais semelhantes
        
        if weights[user_id] < 0:
            continue
            
        clicks_user = clicks_de_filtered[clicks_de_filtered['#User'] == topmatch_users.index[user_id]]['#Offer'] # Clicks de cada usuário da lista top5
        for offer in clicks_user:  # Cliques do usuário
            if offer in clicks_selected_user: # Se já foi um clique do usuário-alvo, não é considerado
                pass
            else:
                recommendations.append((offer, weights[user_id], user_id))
                
    if len(recommendations) == 0: # Caso só existam correlações negativas
        print("Recomendações não foram possíveis")
        return
    
    rec_set = pd.DataFrame(recommendations, columns = ['Offer', 'Weight', 'User']).drop_duplicates() # Dropar Duplicatas
    top5_rec = rec_set.groupby('Offer')['Weight'].sum().sort_values(ascending = False).head() # Top5 Recomendações
    result_final(top5_rec, clicks_selected_user)

In [ ]:
lista_recomendacoes(399566)

In [ ]:
lista_recomendacoes(405012)

In [ ]:
sparse.columns

# Função com base na relação de cliques dos usuarios
- Essa função usa como base a aula de Sistema de Recomendações que tivemos
- Foi feita uma correlação de user_based para o usuario selecionado "aleatoriamente"
- Passos seguidos nessa função:
    - Dataset original foi carregado
    - Renomeação dos usuarios e ofertas
    - Quantificação dos cliques em categorias, usuarios e ofertas
    - Corte minimo de numeros de cliques de categorias, usuarios e ofertas (reduzindo as linhas do dataset)
    - Verificação quantos usuarios tem cliques parecidos com o usuario selecionado
    - Comparação dos demais cliques desses usuarios com o usuario selecionado e recomendação de ofertas que esses usuarios clicaram que o selecionado não clicou

In [ ]:
import pandas as pd
import random

def recomendacao_usuario(path, usuario = 'n',
                         thr_pontuacao_categoria = 10000,
                         thr_pontuacao_oferta = 20,
                         thr_clicks_usuarios = 50,
                        thr_comparacao_usuario_a_usuario = 20,
                        random_seed = 42):
    
    clicks_de = pd.read_csv(path) #carregando o dataframe
    clicks_de.drop(columns = ['Keywords'], axis = 1, inplace=True) #dropando coluna que não será utilizada agora
    #clicks_de['Category'] = clicks_de['Category'].astype(str) #transformando o tipo da coluna em string
    pontuacao_de = pd.DataFrame(clicks_de['Category'].value_counts()).reset_index() #fazendo a pontuação total das categorias
    pontuacao_de.columns = ['Category', 'Pontuacao'] #renomeando as colunas da pontuação

    dici = dict(zip(pontuacao_de.Category, pontuacao_de.Pontuacao)) #transformando em um dicionario para mapear em seguida
    clicks_de['pontuacao'] = clicks_de['Category'].map(dici)
    clicks_de.sort_values(by='pontuacao', inplace = True)

    lista = []
    count = 0
    lista_ids = []


    #renomeando os usuarios em user_1, user_2, etc...
    for coluna in clicks_de.UserId.unique():
        #print(coluna,'_',count)
        #aux = 'user_' + str(count)
        aux = count
        #print(aux.strip())
        lista_ids.append(coluna)
        lista.append(aux)
        count += 1

    lista_offer = []
    count = 0
    lista_offer_ids = []

        #renomeando as ofertas em oferta_1, oferta_2, etc...
    for coluna in clicks_de.OfferId.unique():
        #print(coluna,'_',count)
        #aux = 'oferta_' + str(count)
        aux = count
        #print(aux.strip())
        lista_offer_ids.append(coluna)
        lista_offer.append(aux)
        count += 1

    #mapeando os usuarios + ofertas
    dici_usuarios = dict(zip(lista_ids, lista))
    dici_ofertas = dict(zip(lista_offer_ids, lista_offer))
    clicks_de['usuarios'] = clicks_de['UserId'].map(dici_usuarios)
    clicks_de['ofertas'] = clicks_de['OfferId'].map(dici_ofertas)
    
    clicks_reduzido = clicks_de.drop(columns = ['UserId','OfferId', 'CountryCode','Source','UtcDate', 'OfferViewId'])
    clicks_reduzido = clicks_reduzido.loc[:, ['usuarios','ofertas','Category','pontuacao','OfferTitle']]
    clicks_reduzido.columns = ['usuarios','ofertas','categoria','pontuacao_categoria','titulo_oferta']
    
    
    #criando um esquema de pontuação para cada oferta, quantas vezes aquela oferta foi clicada no total
    pontuacao_ofertas = pd.DataFrame(clicks_reduzido['ofertas'].value_counts()).reset_index()
    pontuacao_ofertas.columns = ['ofertas','pontuacao']
    pontuacao_ofertas = dict(zip(pontuacao_ofertas.ofertas, pontuacao_ofertas.pontuacao))
    clicks_reduzido['pontuacao_ofertas'] = clicks_reduzido['ofertas'].map(pontuacao_ofertas)

    #pontuando os clicks de usuario, quantos clicks cada usuario deu no site
    clicks_usuario =  pd.DataFrame(clicks_reduzido.usuarios.value_counts()).reset_index()
    clicks_usuario.columns = ['usuario','clicks']
    clicks_usuario = dict(zip(clicks_usuario.usuario, clicks_usuario.clicks))
    clicks_reduzido['clicks_usuario'] = clicks_reduzido['usuarios'].map(clicks_usuario)


    #fazendo um corte de tudo, para facilitar a comparação

    mask1 = clicks_reduzido.pontuacao_categoria > thr_pontuacao_categoria
    clicks_reduzido = clicks_reduzido.loc[mask1, :]
    mask2 = clicks_reduzido.pontuacao_ofertas > thr_pontuacao_oferta
    clicks_reduzido = clicks_reduzido.loc[mask2,:]
    mask3 = clicks_reduzido.clicks_usuario > thr_clicks_usuarios
    clicks_reduzido = clicks_reduzido.loc[mask3, :]
    
    sparse = clicks_reduzido.pivot_table(values='clicks_usuario',index='usuarios',columns='ofertas').T

    sparse[sparse.isna()] = 0
    sparse[sparse != 0] = 1


    sparse_user = sparse.T

    random.seed(random_seed)
    if usuario == 'n':
        #usuario = random.choice(sparse_user.T.index)
        usuario = random.choice(clicks_reduzido.usuarios.unique())

    #mostrando os itens que o usuario selecionado clicou
    items_users = clicks_reduzido.loc[clicks_reduzido.usuarios == int(usuario),'ofertas'].dropna()

    #listando todos os usuarios e guardando em uma lista que será usada no looping
    list_of_users = clicks_reduzido.usuarios.unique().tolist()

    #variavel que vai selecionar só os usuarios que tiverem mais que o numero desejado de cliques iguais. Configurado como base em 20!
    thr_comparable = thr_comparacao_usuario_a_usuario

    comparable_users = []

    for user_loop in list_of_users:
        if clicks_reduzido.loc[clicks_reduzido.usuarios == user_loop,'ofertas'].dropna().isin(items_users).sum() > thr_comparable:
            comparable_users.append(user_loop)

            
            
    #selecionando os 5 usuarios mais similares ao nosso usuario target
    top5_most_similar = sparse.loc[:,comparable_users].corrwith(sparse.loc[:,usuario]).sort_values(ascending=False).index[1:6]


    suggested_user = pd.DataFrame()



    #nesse loop adicionamos todos os clicks dos usuarios parecidos que nosso usuario selecionado não tenha clicado!
    for user in top5_most_similar:
        #print(user)
        clicks_user = clicks_reduzido.loc[clicks_reduzido.usuarios == user, :].dropna()
        clicks_user = clicks_user.loc[~clicks_user.ofertas.isin(items_users)]
        #clicks_user = clicks_user.drop_duplicates(subset = 'ofertas')
        suggested_user = pd.concat([suggested_user, clicks_user])
        #print(suggested_user.shape)



        #fazemos uma nova pontuação, com base só nos cliques dos usuarios parecidos e retornamos um dataframe ordenado por essa pontuacao

    pt = suggested_user['ofertas'].value_counts().reset_index()
    pt.columns = ['oferta','pontuacao_ofertas_usuarios']
    nova_pontuacao = dict(zip(pt.oferta, pt.pontuacao_ofertas_usuarios))
    suggested_user['nova_pontuacao'] = suggested_user.ofertas.map(nova_pontuacao)
    suggested_user.drop(columns=['pontuacao_ofertas'], inplace = True)
    suggested_user.drop_duplicates(subset='ofertas', inplace = True)
    suggested_user.sort_values(by='nova_pontuacao', ascending = False, inplace = True)

    dicionario_ofertas_titulos = dict(zip(clicks_reduzido.ofertas, clicks_reduzido.titulo_oferta))
    items_users = pd.DataFrame(items_users)
    items_users['titulo_oferta'] = items_users.ofertas.map(dicionario_ofertas_titulos)
    items_users['usuarios'] = usuario
    #items_users = items_users.loc[:,['usuario', 'ofertas','titulo_oferta']]
           
    return (usuario, items_users, suggested_user)

In [ ]:
%%time
path = 'clicks_de_sample_2.txt'
usuario, item_do_usuario, sugestoes = recomendacao_usuario(path)

In [ ]:
print('usuario selecionado: ',usuario)
item_do_usuario.head(10)

In [ ]:
sugestoes.head(20)

# Funções para obtenção da categoria do

In [ ]:
## 601 é a homepage, se aplica a todos
## Primeira separação de categorias
    
def categories(df_cat):
    listados = []
    for item in df_cat.Ancestor_ID:
        listados.append(item.split())
    categories = list()
    for cats in range(1,4):
        category = list()
        for item in listados:
            if len(item) == cats+1:
                category.append(item) 
        categories.append(category)
    return categories

categories = categories(df_de_cat)

In [ ]:
macro4 = categories[2]
macro4

In [ ]:
df4 = pd.DataFrame(macro4)#.drop(3, axis = 1)
df4.head()

In [ ]:
macro3 = categories[1]
df3 = pd.DataFrame(macro3, columns=(1,2,3))
df3.head()

In [ ]:
macro2 = categories[0]
df2 = pd.DataFrame(macro2, columns = (2,3))
df2.head()

In [ ]:
new_df = pd.concat((df4,df3,df2), axis = 0).reset_index(drop = True)
new_df

In [ ]:
grupos1 = list(new_df.dropna(axis = 0).groupby(1)[0])
grupos1

In [ ]:
keys = [keys[0] for keys in grupos1]
itens = [itens[1].tolist() for itens in grupos1]
dicio_cat1 = dict(zip(keys, itens))
dicio_cat1

In [ ]:
def getcat1(produto):
    for key, value in dicio1.items():
        if str(produto) in value:
            return ("categ0: ", produto, " categ1: ", key)
    return False

In [ ]:
getcat1('125601')

In [ ]:
grupos2 = list(new_df.drop(0, axis = 1).dropna(axis=0).groupby(2)[1])
keys = [keys[0] for keys in grupos2]
itens = [itens[1].unique().tolist() for itens in grupos2]
dicio_cat2 = dict(zip(keys, itens))

In [ ]:
dicio_cat2

In [ ]:
def getcat2(produto):
    categoria = getcat1(produto)
    if categoria is not False:
        for key, value in dicio_cat2.items():
            if str(categoria) in value:
                return key, categoria
        return False
    else:
        for key, value in dicio_cat2.items():
            if str(produto) in value:
                return ('categ1: ', key,' categ2: ', produto)
        return False


In [ ]:
getcat2('173801')

In [ ]:
tradu_cat = dict(zip(df_de_cat.Ancestor_ID.tolist(), df_de_cat.Name.tolist())) # Dicionário de Tradução de categorias
tradu_cat

## Considerações:

- Problema de Cold Start
- Limitação de recomendação apenas de itens já clicados por outros usuários
- Limitação de atributos dos usuários e produtos

## Próximos Passos:

- Utilizar as categorias, supercategorias e subcategorias dos itens clicados para fazer novas recomendações com base nesses agrupamentos
- Utilizar o horário e sequencia dos clicks para fazer novas recomendações
- Agregar previsões/recomendações a partir de algoritmos de Machine Learning (KNN, etc) ***PRIORIZAR***
- Avaliar a clusterização de usuários para definir personas dos clientes
- Avaliar a implementação de Ensembles (Voting Classifier) utilizando diferentes estratégias para definição das recomendações
- Simulação com usuários aleatórios.


In [1]:
from surprise import Dataset, Reader
import os

In [2]:
reader = Reader(line_format='user item rating', sep=',', rating_scale = (0,1))

In [3]:
data = Dataset.load_from_file(file_path ='user_item_clicks', reader=reader)